<a href="https://colab.research.google.com/github/martinbedia/mineria/blob/main/ejercicio%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 2
Crea un nuevo programa para procesar el nuevo formato de ticket para obtener un archivo csv en el que se muestre el ID del ticket, producto y precio. Genera también un segundo csv donde para cada ticket se muestre la dirección del establecimiento, la hora de compra, el total del ticket, el descuento generado y el acumulado.

In [5]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00


In [1]:
!pip install pymupdf pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 34.7 MB/s eta 0:00:00


In [2]:
!pip install pytesseract

In [3]:
!pip install Pillow

In [6]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import KMeans, AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns
import glob, os
import PyPDF2
import re
import csv
from datetime import datetime
import fitz  # PyMuPDF
from PIL import Image
import pytesseract

In [7]:
# leer archivos de un tipo concreto de una carpeta y sus subcarpetas
# recursive para que busque en las subcarpetas
ruta_carpeta_raiz = '.'
tickets = glob.glob(os.path.join(ruta_carpeta_raiz, '**/*.pdf'),recursive=True)
tickets

['./Ticket-23_11_2023.pdf']

In [8]:
# leer el ticket
pdf_reader = PyPDF2.PdfReader(tickets[0])    # conectar el archivo pdf con el codigo en python
ticket = pdf_reader.pages[0]   # acceder a la primera pagina
texto_ticket = ticket.extract_text()
texto_ticket

''

En este caso, la función para leer los tickets del ejercicio anterior no funciona ya que el texto del ticket no aparece como texto, sino como una imagen dentro del pdf (dando como resultado vacío al usarla). Trataremos de leer el pdf mediante un metodo que nos permita extraer el texto de las imagenes contenidas en el pdf.

In [9]:
def extract_text_from_pdf(pdf_path):
    # Abrir el archivo PDF
    pdf_document = fitz.open(pdf_path)

    # Iterar sobre las páginas del PDF
    text = ""
    for page_number in range(pdf_document.page_count):
        # Obtener la página actual
        page = pdf_document[page_number]

        # Convertir la página a una imagen
        image = page.get_pixmap()

        # Guardar la imagen en un archivo temporal
        image_path = f"temp_page_{page_number + 1}.png"
        image.save(image_path)

        # Utilizar pytesseract para realizar OCR en la imagen
        page_text = pytesseract.image_to_string(Image.open(image_path))

        # Agregar el texto de la página al resultado final
        text += page_text + "\n"

    # Cerrar el archivo PDF
    pdf_document.close()

    return text

# Ruta del archivo PDF
pdf_path = "/content/Ticket-23_11_2023.pdf"

# Extraer texto del PDF
texto_ticket = extract_text_from_pdf(pdf_path)

# Imprimir el texto extraído
print(texto_ticket)


TesseractNotFoundError: ignored

In [10]:
def extract_text_from_pdf(pdf_path):
    text = ""

    # Abrir el archivo PDF
    pdf_document = fitz.open(pdf_path)

    # Iterar sobre las páginas del PDF
    for page_number in range(pdf_document.page_count):
        # Obtener la página actual
        page = pdf_document[page_number]

        # Extraer texto de la página
        text += page.get_text()

    # Cerrar el archivo PDF
    pdf_document.close()

    return text

# Ruta del archivo PDF
pdf_path = "/content/Ticket-23_11_2023.pdf"

# Extraer texto del PDF
texto_ticket = extract_text_from_pdf(pdf_path)

# Imprimir el texto extraído
print(texto_ticket)


Con esas funciones deberíamos poder extraer el texto de al menos una manera, pero me sale un error y no puedo usar el paquete tesseract (TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information).
De todas maneras, las lineas siguientes se deberian poder utilizar para extraer la informacion del texto (a falta de poder extraer el propio texto del pdf)

In [12]:
# texto del ticket por lineas
lineas = np.array(texto_ticket.split('\n'))
lineas

array([''], dtype='<U1')

In [53]:
def leer_lineas_de_pdf (nombre_del_pdf):
  pdf_reader = PyPDF2.PdfReader(nombre_del_pdf)
  ticket = pdf_reader.pages[0]
  texto_ticket = ticket.extract_text()
  lineas = np.array(texto_ticket.split('\n'))
  return lineas

In [61]:
lineas_ticekt1 = leer_lineas_de_pdf('/content/20230909 Mercadona 21,23 €.pdf')
lineas_ticekt1

array(['MERCADONA, S.A.   A-46103834', 'PLAZA DAOIZ Y VELARDE 5',
       '33009 OVIEDO', 'TELÉFONO: 985216153',
       '09/09/2023 20:35  OP: 191484',
       'FACTURA SIMPLIFICADA: 3433-011-051093',
       'Descripción P. Unit Importe', '2TAQUITOS SALMÓN 3,80 7,60',
       '1TROTTOLE TRICOLORE 0,95', '1QUESO RALLADO GOUDA 1,44',
       '1ÑOQUIS DE PATATA 1,12', '2LECHE SEMI S/LACT 1,46 2,92',
       '1QUESO FETA 2,35', '1PICATOSTE AJO 0,85',
       '2NEGRA RODAJAS P3 2,00 4,00', '1PARKING 0,00',
       'ENTRADA  20:22       SALIDA  20:35', 'TOTAL (€) 21,23',
       'TARJETA BANCARIA 21,23', 'IVA BASE IMPONIBLE (€) CUOTA (€)',
       '10% 11,32 1,13', '5% 1,97 0,10', '0% 6,71 0,00',
       'TOTAL 20,00 1,23', 'TARJ. BANCARIA:  **** **** **** 4016',
       'N.C: 003943818  AUT: 729144', 'AID: A0000000041010   ARC: 3030',
       'MASTERCARD', 'Importe: 21,23 € MASTERCARD',
       'SE ADMITEN DEVOLUCIONES CON TICKET', 'DISPONE DE 20 MINUTOS',
       'PARA RETIRAR SU VEHÍCULO'], dtype='<U37

In [62]:
def extraer_id_ticket(lineas):
    id_ticket_match = re.search(r'\d*-\d*-\d*', '\n'.join(lineas))
    id_ticket = id_ticket_match.group() if id_ticket_match else None

    return id_ticket

extraer_id_ticket(lineas_ticekt1)

'3433-011-051093'

In [63]:
def extraer_productos(lineas):
  productos = []
  for linea in lineas:
        producto_match = re.search(r'\d[A-ZÑñÁÉÍÓÚáéíóú][[A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*((\d{1,2},\d{1,2}\s*){1,2})', linea, flags=re.IGNORECASE)
        if producto_match:
            productos.append(producto_match.group())
  return productos

productos_ticket = extraer_productos(lineas_ticekt1)
productos_ticket


['2TAQUITOS SALMÓN 3,80 7,60',
 '1TROTTOLE TRICOLORE 0,95',
 '1QUESO RALLADO GOUDA 1,44',
 '1ÑOQUIS DE PATATA 1,12',
 '2LECHE SEMI S/LACT 1,46 2,92',
 '1QUESO FETA 2,35',
 '1PICATOSTE AJO 0,85',
 '2NEGRA RODAJAS P3 2,00 4,00',
 '1PARKING 0,00']

In [64]:
def obtener_nombre_producto(linea):
    # Buscar coincidencias en la línea
    coincidencia = re.search(r'^\d{1,}([A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*\s)', linea, flags=re.IGNORECASE)

    if coincidencia:
        return coincidencia.group(1).strip()
    else:
        return None

# Obtener los nombres de los productos
nombres_productos = [obtener_nombre_producto(linea) for linea in productos_ticket]

print(nombres_productos)

['TAQUITOS SALMÓN', 'TROTTOLE TRICOLORE', 'QUESO RALLADO GOUDA', 'ÑOQUIS DE PATATA', 'LECHE SEMI S/LACT', 'QUESO FETA', 'PICATOSTE AJO', 'NEGRA RODAJAS P3', 'PARKING']


In [65]:
def obtener_precio_producto(linea):
    # Buscar coincidencias en la línea
    coincidencia = re.search(r'(\d{1,2},\d{1,2})', linea)

    if coincidencia:
        return coincidencia.group(1)
    else:
        return None

# Obtener los precios de los productos
precios_productos = [obtener_precio_producto(linea) for linea in productos_ticket]

print(precios_productos)

['3,80', '0,95', '1,44', '1,12', '1,46', '2,35', '0,85', '2,00', '0,00']


In [66]:
# todo en la misma funcion
ruta_carpeta_raiz = '.'
tickets = glob.glob(os.path.join(ruta_carpeta_raiz, '**/*.pdf'),recursive=True)
tickets

def ejercicio2(tickets):
  # leer el ticket
  pdf_reader = PyPDF2.PdfReader(tickets)    # conectar el archivo pdf con el codigo en python
  ticket = pdf_reader.pages[0]   # acceder a la primera pagina
  texto_ticket = ticket.extract_text()
  # texto del ticket por lineas
  lineas = np.array(texto_ticket.split('\n'))

  id_ticket_match = re.search(r'TICKET:\s\d+', '\n'.join(lineas))
  id_ticket = id_ticket_match.group() if id_ticket_match else None

  productos = []
  for linea in lineas:
        producto_match = re.search(r'\d[A-ZÑñÁÉÍÓÚáéíóú][[A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*((\d{1,2},\d{1,2}\s*){1,2})', linea, flags=re.IGNORECASE)
        if producto_match:
            productos.append(producto_match.group())


  # Buscar coincidencias en el nombre
  nombres = []
  for linea in productos:
    nombres_matches = re.search(r'^\d{1,}([A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*\s)', linea, flags=re.IGNORECASE)
    nombres.append(nombres_matches.group(1).strip() if nombres_matches else None)

  # Buscar coincidencias en el precio
  precios = []
  for linea in productos:
    precio_matches = re.search(r'(\d{1,2},\d{1,2})', linea)
    precios.append(precio_matches.group(1) if precio_matches else None)

  df = pd.DataFrame({
    'id ticket': id_ticket,
    'producto': nombres,
    'precio_individual': precios
  })


  return df

In [67]:
ejercicio2('/content/20230909 Mercadona 21,23 €.pdf')

,id ticket,producto,precio_individual
0,3433-011-051093,TAQUITOS SALMÓN,"3,80"
1,3433-011-051093,TROTTOLE TRICOLORE,"0,95"
2,3433-011-051093,QUESO RALLADO GOUDA,"1,44"
3,3433-011-051093,ÑOQUIS DE PATATA,"1,12"
4,3433-011-051093,LECHE SEMI S/LACT,"1,46"
5,3433-011-051093,QUESO FETA,"2,35"
6,3433-011-051093,PICATOSTE AJO,"0,85"
7,3433-011-051093,NEGRA RODAJAS P3,"2,00"
8,3433-011-051093,PARKING,"0,00"


In [68]:
# funcion de los descuentos
def ejercicio2_descuento(tickets):
  # leer el ticket (usaremos el codigo del ejercicio 1, el cual deberiamos cambiar por el de leer los tickets de la nueva forma)
  pdf_reader = PyPDF2.PdfReader(tickets)
  ticket = pdf_reader.pages[0]
  texto_ticket = ticket.extract_text()
  # texto del ticket por lineas
  lineas = np.array(texto_ticket.split('\n'))

  id_ticket_match = re.search(r'TICKET:\s\d+', '\n'.join(lineas))
  id_ticket = id_ticket_match.group() if id_ticket_match else None

  # Buscar la presencia de "PARKING" en el ticket
  parking_match = re.search(r'1PARKING', '\n'.join(lineas), flags=re.IGNORECASE)
  # Determinar si el ticket contiene "1PARKING"
  contiene_parking = True if parking_match else False

  textos_tickets = []
  direcciones = []
  direccion = lineas[3] if len(lineas) > 3 else None
  direcciones.append(direccion)

  precio_match = re.search(r'TOTAL.*\s\d{1,2},\d{1,2}', '\n'.join(lineas))
  precio_total = precio_match.group() if precio_match else None
  total_match = re.search(r'\d{1,2},\d{1,2}', precio_total)
  total_pagado = total_match.group() if total_match else None

  coincidencia_hora = re.search(r'HORA.\s(\d{2}:\d{2}:\d{2})', '\n'.join(lineas), flags=re.IGNORECASE)
  horas_entrada, horas_salida = coincidencia_hora.groups() if coincidencia_hora else (None, None)

  descuento_gen_matches = re.search(r'Euros conseguidos.*\s\d{1,2},\d{1,2}', '\n'.join(lineas))
  descuento_gen = descuento_gen_matches.group() if descuento_gen_matches else None

  descuento_ac_matches = re.search(r'Saldo próximo Vale Mensual.*\s\d{1,2},\d{1,2}', '\n'.join(lineas))
  descuento_ac = descuento_ac_matches.group() if descuento_ac_matches else None

  df = pd.DataFrame({
    'id ticket': id_ticket,
    'direccion': direcciones,
    'Hora': horas_entrada,
    'total precio': total_pagado,
    'Descuento Generado': descuento_gen,
    'Descuento Acumulado': descuento_ac
  },index=[0])

  return df

In [69]:
ejercicio2_descuento('/content/20230909 Mercadona 21,23 €.pdf')

,id ticket,direccion,Hora,total precio,Descuento Generado,Descuento Acumulado
0,3433-011-051093,PLAZA DAOIZ Y VELARDE 5,20:22,"21,23",None,None


Finalmente, ponemos todas las partes anteriores en una misma funcion que nos devuelva directamente los csv que nos pide el enunciado.

In [ ]:
def ejercicio2_final(tickets):
  # leer el ticket
  pdf_reader = PyPDF2.PdfReader(tickets)    # conectar el archivo pdf con el codigo en python
  ticket = pdf_reader.pages[0]   # acceder a la primera pagina
  texto_ticket = ticket.extract_text()
  # texto del ticket por lineas
  lineas = np.array(texto_ticket.split('\n'))

  id_ticket_match = re.search(r'TICKET:\s\d+', '\n'.join(lineas))
  id_ticket = id_ticket_match.group() if id_ticket_match else None

  productos = []
  for linea in lineas:
        producto_match = re.search(r'\d[A-ZÑñÁÉÍÓÚáéíóú][[A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*((\d{1,2},\d{1,2}\s*){1,2})', linea, flags=re.IGNORECASE)
        if producto_match:
            productos.append(producto_match.group())

  # Buscar coincidencias en el nombre
  nombres = []
  for linea in productos:
    nombres_matches = re.search(r'^\d{1,}([A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*\s)', linea, flags=re.IGNORECASE)
    nombres.append(nombres_matches.group(1).strip() if nombres_matches else None)

  # Buscar coincidencias en el precio
  precios = []
  for linea in productos:
    precio_matches = re.search(r'(\d{1,2},\d{1,2})', linea)
    precios.append(precio_matches.group(1) if precio_matches else None)

  # Buscar la presencia de "PARKING" en el ticket
  parking_match = re.search(r'1PARKING', '\n'.join(lineas), flags=re.IGNORECASE)
  # Determinar si el ticket contiene "1PARKING"
  contiene_parking = True if parking_match else False

  textos_tickets = []
  direcciones = []
  direccion = lineas[3] if len(lineas) > 3 else None
  direcciones.append(direccion)

  precio_match = re.search(r'TOTAL.*\s\d{1,2},\d{1,2}', '\n'.join(lineas))
  precio_total = precio_match.group() if precio_match else None
  total_match = re.search(r'\d{1,2},\d{1,2}', precio_total)
  total_pagado = total_match.group() if total_match else None

  coincidencia_hora = re.search(r'HORA.\s(\d{2}:\d{2}:\d{2})', '\n'.join(lineas), flags=re.IGNORECASE)
  horas_entrada, horas_salida = coincidencia_hora.groups() if coincidencia_hora else (None, None)

  descuento_gen_matches = re.search(r'Euros conseguidos.*\s\d{1,2},\d{1,2}', '\n'.join(lineas))
  descuento_gen = descuento_gen_matches.group() if descuento_gen_matches else None

  descuento_ac_matches = re.search(r'Saldo próximo Vale Mensual.*\s\d{1,2},\d{1,2}', '\n'.join(lineas))
  descuento_ac = descuento_ac_matches.group() if descuento_ac_matches else None

  df1 = pd.DataFrame({
    'id ticket': id_ticket,
    'direccion': direcciones,
    'Hora': horas_entrada,
    'total precio': total_pagado,
    'Descuento Generado': descuento_gen,
    'Descuento Acumulado': descuento_ac
  },index=[0])

  df2 = pd.DataFrame({
    'id ticket': id_ticket,
    'producto': nombres,
    'precio_individual': precios
  })

  df1_csv = df1.to_csv('df1_csv', index= False)
  df2_csv = df1.to_csv('df2_csv', index= False)

  return df1_csv, df2_csv